In [50]:
# Install required packages in the folllowing few cells

In [53]:
%%capture
 # capture hides output
!pip install yfinance pandas numpy scikit-learn ta pandas_ta xgboost

In [47]:
%%capture
# hidden output
!pip install alpha_vantage

In [48]:
%%capture
# hidden output, !pip install is required to run just on a machine
!pip install requests pandas_datareader


#### IMPORTS

In [17]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json


from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


### ALPHA VANTAGE DATA

In [33]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=TSLA&apikey='+os.getenv('AV_API_KEY')
r = requests.get(url)
data = r.json()

print(json.dumps(data,indent=4))

{
    "symbol": "TSLA",
    "annualReports": [
        {
            "fiscalDateEnding": "2023-12-31",
            "reportedCurrency": "USD",
            "operatingCashflow": "13256000000",
            "paymentsForOperatingActivities": "2376000000",
            "proceedsFromOperatingActivities": "None",
            "changeInOperatingLiabilities": "4137000000",
            "changeInOperatingAssets": "4433000000",
            "depreciationDepletionAndAmortization": "3836000000",
            "capitalExpenditures": "8898000000",
            "changeInReceivables": "586000000",
            "changeInInventory": "1195000000",
            "profitLoss": "14974000000",
            "cashflowFromInvestment": "-15584000000",
            "cashflowFromFinancing": "2589000000",
            "proceedsFromRepaymentsOfShortTermDebt": "None",
            "paymentsForRepurchaseOfCommonStock": "None",
            "paymentsForRepurchaseOfEquity": "None",
            "paymentsForRepurchaseOfPreferredStock": "No

In [34]:
#json file, we can extract detailed data we need for a company
data["annualReports"]

[{'fiscalDateEnding': '2023-12-31',
  'reportedCurrency': 'USD',
  'operatingCashflow': '13256000000',
  'paymentsForOperatingActivities': '2376000000',
  'proceedsFromOperatingActivities': 'None',
  'changeInOperatingLiabilities': '4137000000',
  'changeInOperatingAssets': '4433000000',
  'depreciationDepletionAndAmortization': '3836000000',
  'capitalExpenditures': '8898000000',
  'changeInReceivables': '586000000',
  'changeInInventory': '1195000000',
  'profitLoss': '14974000000',
  'cashflowFromInvestment': '-15584000000',
  'cashflowFromFinancing': '2589000000',
  'proceedsFromRepaymentsOfShortTermDebt': 'None',
  'paymentsForRepurchaseOfCommonStock': 'None',
  'paymentsForRepurchaseOfEquity': 'None',
  'paymentsForRepurchaseOfPreferredStock': 'None',
  'dividendPayout': 'None',
  'dividendPayoutCommonStock': 'None',
  'dividendPayoutPreferredStock': 'None',
  'proceedsFromIssuanceOfCommonStock': 'None',
  'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet': 'None',
  'pr

In [ ]:
# Function to get stock data from Alpha Vantage

def getalphastock(api_key, stockname='msft',timeint='1min'):

    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_intraday(symbol=stockname,interval=timeint,outputsize='full')
    return data

In [54]:
import yfinance as yf
from pandas_datareader import data as pdr
# Function to get stock data from Yahoo Finance

def getyahooticks(tickersin="AAPL MSFT", periodin = "1d", intervalin="5m"):
    yf.pdr_override()

    data = pdr.get_data_yahoo(
            # tickers list or string as well
            tickers = tickersin,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (default is '1mo')
            period = periodin,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = intervalin,

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = True,

            # download pre/post regular market hours data
            # (optional, default is False)
            prepost = True,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            proxy = None
        )
    return data

In [41]:

# Load environment variables from .env file
load_dotenv()

# Access your API key from the environment variables
api_key = os.getenv('AV_API_KEY')

#Get Data
ts = TimeSeries(key=api_key, output_format='pandas')

stockname = 'MSFT'
period = 5
freqfix = '5min'
datainter = '5m'

#pick data source
fromdata='alpha'

if fromdata == 'alpha':
    data = getalphastock(api_key,stockname,'5min')
    stockpricefull = getalphastock(api_key,stockname)
    stockpricefull.columns = ['Open','High','Low','Close','Volume']
    stock = data.copy()
    stock.columns = ['Open','High','Low','Close','Volume']
    stockpricefull.head()

elif fromdata=='yahoo':
    stock = getyahooticks(tickersin=stockname, periodin = "5d", intervalin=datainter)
    #stockpricefull = getyahooticks(tickersin=stockname, periodin = "30d", intervalin="5m")
    stockpricefull= stock.copy()
    stock.index.name='date'
    stockpricefull.index.name='date'
    stock.index = stock.index.tz_localize(None)#.tz_convert(tz = None)
    stockpricefull.index = stockpricefull.index.tz_localize(None)
    #stockpricefull.index.to_datetime

print(stockpricefull)

                       Open    High     Low   Close  Volume
date                                                       
2024-04-25 19:59:00  416.44  416.70  416.20  416.65  4374.0
2024-04-25 19:58:00  416.30  416.50  416.15  416.40  1682.0
2024-04-25 19:57:00  416.15  416.40  416.01  416.30  1978.0
2024-04-25 19:56:00  416.29  416.30  415.92  416.13  1284.0
2024-04-25 19:55:00  415.75  416.20  415.60  416.09  1721.0
...                     ...     ...     ...     ...     ...
2024-04-04 04:04:00  422.14  422.14  422.14  422.14     2.0
2024-04-04 04:03:00  422.14  422.60  422.14  422.17   297.0
2024-04-04 04:02:00  422.86  422.86  422.14  422.53   736.0
2024-04-04 04:01:00  422.89  422.89  422.74  422.74    85.0
2024-04-04 04:00:00  424.26  424.26  422.10  422.56   549.0

[15138 rows x 5 columns]
